<a href="https://colab.research.google.com/github/renatachavezm22/Regresion_Lineal/blob/main/7_Regresi%C3%B3n_Lineal_Multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Cargar archivo csv desde equipo
#from google.colab import files
#files.upload()

In [ ]:
df= pd.read_csv('California_EUA.csv', encoding = 'unicode_escape')
df2 = pd.read_csv('DF_Mexico.csv')
df3 = pd.read_csv('Girona_España.csv', encoding = 'unicode_escape')

df = df.dropna(how = 'all',axis = 1)
df2 = df2.dropna(how = 'all',axis = 1)
df3 = df3.dropna(how = 'all',axis = 1)

numeric_cols = df.select_dtypes(include=['int', 'float'])
numeric_cols = list(numeric_cols.columns)
numeric_cols

for i in numeric_cols:
  df[i] = df[i].fillna(round(df[i].median(),2))

numeric_cols2 = df2.select_dtypes(include=['int', 'float'])
numeric_cols2 = list(numeric_cols2.columns)
numeric_cols2

for i in numeric_cols2:
  df2[i] = df2[i].fillna(round(df2[i].median(),2))

numeric_cols3 = df3.select_dtypes(include=['int', 'float'])
numeric_cols3 = list(numeric_cols3.columns)
numeric_cols3

for i in numeric_cols3:
  df3[i] = df3[i].fillna(round(df3[i].median(),2))

df = df.fillna(method='ffill')
df2 = df2.fillna(method='ffill')
df3 = df3.fillna(method='ffill')

df = df.fillna(method='bfill')
df2 = df2.fillna(method='bfill')
df3 = df3.fillna(method='bfill')

y = df[numeric_cols]

percentile25 = y.quantile(0.25) #Q1
percentile75 = y.quantile(0.75) #Q3
iqr = percentile75 - percentile25

Limite_Superior = percentile75 + 1.5*iqr
Limite_Inferior = percentile25 - 1.5*iqr

data_clean1 = y[(y<=Limite_Superior)&(y>=Limite_Inferior)]
df = df.drop(numeric_cols[1:], axis =1)
df_1 = pd.merge(df, data_clean1)
df_1

y = df2[numeric_cols2]

percentile25 = y.quantile(0.25) #Q1
percentile75 = y.quantile(0.75) #Q3
iqr = percentile75 - percentile25

Limite_Superior = percentile75 + 1.5*iqr
Limite_Inferior = percentile25 - 1.5*iqr

data_clean2 = y[(y<=Limite_Superior)&(y>=Limite_Inferior)]
df2 = df2.drop(numeric_cols2[1:], axis =1)
df_2 = pd.merge(df2, data_clean2)
df_2

y = df3[numeric_cols3]

percentile25 = y.quantile(0.25) #Q1
percentile75 = y.quantile(0.75) #Q3
iqr = percentile75 - percentile25

Limite_Superior = percentile75 + 1.5*iqr
Limite_Inferior = percentile25 - 1.5*iqr

data_clean3 = y[(y<=Limite_Superior)&(y>=Limite_Inferior)]
df3 = df3.drop(numeric_cols3[1:], axis =1)
df_3 = pd.merge(df3, data_clean3)

<ipython-input-34-47c21c215565>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[i] = df3[i].fillna(round(df3[i].median(),2))
<ipython-input-34-47c21c215565>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[i] = df3[i].fillna(round(df3[i].median(),2))
<ipython-input-34-47c21c215565>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [ ]:
datasets = [df_1,df_2,df_3]

for i in datasets: 
  i.price= i.price.replace({'\$':''}, regex = True)
  i.price= i.price.replace({'\,':''}, regex = True)
  i['price'] = i['price'].astype(float)

  i.host_acceptance_rate= i.host_acceptance_rate.replace({'%':''}, regex = True)
  i['host_acceptance_rate'] = i['host_acceptance_rate'].astype(float)

  i.host_response_rate= i.host_response_rate.replace({'%':''}, regex = True)
  i['host_response_rate'] = i['host_response_rate'].astype(float)

  i.fillna(method = 'bfill')
  i.fillna(method = 'ffill')
  i = i.dropna(how='any',axis =0)

##Creación de filtros

In [ ]:
cal_1 = df_1[df_1['room_type'] == 'Entire home/apt']
cal_2 = df_1[df_1['room_type'] == 'Private room']

DF_1 = df_2[df_2['room_type'] == 'Entire home/apt']
DF_2 = df_2[df_2['room_type'] == 'Private room']

gir_1 = df_3[df_3['room_type'] == 'Entire home/apt']
gir_2 = df_3[df_3['room_type'] == 'Private room']

##Creación de modelos para california

In [ ]:
#Entire home/apt
Vars1_Indep= cal_1[['host_response_rate', 'price', 'number_of_reviews','review_scores_rating']]
Var1_Dep= cal_1['host_acceptance_rate']

Vars1_Indep2= cal_1[['availability_365','host_response_rate', 'number_of_reviews', 'accommodates']]
Var1_Dep2= cal_1['price']

Vars1_Indep3= cal_1[['number_of_reviews','review_scores_cleanliness', 'review_scores_communication']]
Var1_Dep3= cal_1['host_acceptance_rate']

Vars2_Indep= cal_2[['host_response_rate', 'price', 'number_of_reviews','review_scores_rating']]
Var2_Dep= cal_2['host_acceptance_rate']

Vars2_Indep2= cal_2[['availability_365','host_response_rate', 'number_of_reviews', 'accommodates']]
Var2_Dep2= cal_2['price']

Vars2_Indep3= cal_2[['number_of_reviews','review_scores_cleanliness', 'review_scores_communication']]
Var2_Dep3= cal_2['host_acceptance_rate']

In [ ]:
#Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
model= LinearRegression()

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
var_ind = [Vars1_Indep, Vars1_Indep2, Vars1_Indep3, Vars2_Indep, Vars2_Indep2, Vars2_Indep3]
var_dep = [Var1_Dep, Var1_Dep2, Var1_Dep3, Var2_Dep, Var2_Dep2, Var2_Dep3]

model_dict = []
det = []
corr = []

for i in range(len(var_ind)):
  var_ind[i] = var_ind[i].fillna(method = 'bfill')
  var_ind[i] = var_ind[i].fillna(method = 'ffill')
  var_dep[i] = var_dep[i].fillna(method = 'bfill')
  var_dep[i] = var_dep[i].fillna(method = 'ffill')
  model.fit(X = var_ind[i], y = var_dep[i])
  det.append(model.score(var_ind[i], var_dep[i]))
  corr.append(np.sqrt(model.score(var_ind[i], var_dep[i])))
  model_dict.append(model.__dict__)
  x = model_dict[i]
  print(x['coef_'].max(),'x + ', x['intercept_'])

print()
display(det, corr)

3.358121725976674 x +  39.256928735536015
53.96717642428753 x +  50.44329724351036
2.554548866064342 x +  66.46318622220873
0.11302386803660218 x +  133.01498416399852
34.89277232601321 x +  1137.676527246565
0.08089446215558876 x +  209.36463989627006



[0.05909374065447426,
 0.08071142968801426,
 0.010242139071000356,
 0.04183257677146246,
 0.017392827861884896,
 0.02520929996361465]

[0.24309204152845945,
 0.28409757071825564,
 0.10120345384916642,
 0.20453013658496016,
 0.1318818708613314,
 0.1587743680938918]

In [ ]:
cali_1 = []
cali_1 = pd.DataFrame(cali_1)
cali_1['Determinacion'] = det[0:6]
cali_1['Correlacion'] = corr[0:6]

cali_2 = []
cali_2 = pd.DataFrame(cali_2)
cali_2['Determinacion'] = det[6:]
cali_2['Correlacion'] = corr[6:]

cali = []
cali = pd.DataFrame(cali)
cali['Determinacion'] = det
cali['Correlacion'] = corr

display(cali_1,cali_2, cali)

,Determinacion,Correlacion
0,0.059094,0.243092
1,0.080711,0.284098
2,0.010242,0.101203
3,0.041833,0.204530
4,0.017393,0.131882
5,0.025209,0.158774


,Determinacion,Correlacion


,Determinacion,Correlacion
0,0.059094,0.243092
1,0.080711,0.284098
2,0.010242,0.101203
3,0.041833,0.204530
4,0.017393,0.131882
5,0.025209,0.158774


##Creación de modelos para D.F.

In [ ]:
#Entire home/apt
Vars1_Indep= DF_1[['host_response_rate', 'price', 'number_of_reviews','review_scores_rating']]
Var1_Dep= DF_1['host_acceptance_rate']

Vars1_Indep2= DF_1[['availability_365','host_response_rate', 'number_of_reviews', 'accommodates']]
Var1_Dep2= DF_1['price']

Vars1_Indep3= DF_1[['number_of_reviews','review_scores_cleanliness', 'review_scores_communication']]
Var1_Dep3= DF_1['host_acceptance_rate']

Vars2_Indep= DF_2[['host_response_rate', 'price', 'number_of_reviews','review_scores_rating']]
Var2_Dep= DF_2['host_acceptance_rate']

Vars2_Indep2= DF_2[['availability_365','host_response_rate', 'number_of_reviews', 'accommodates']]
Var2_Dep2= DF_2['price']

Vars2_Indep3= DF_2[['number_of_reviews','review_scores_cleanliness', 'review_scores_communication']]
Var2_Dep3= DF_2['host_acceptance_rate']

In [ ]:
var_ind = [Vars1_Indep, Vars1_Indep2, Vars1_Indep3, Vars2_Indep, Vars2_Indep2, Vars2_Indep3]
var_dep = [Var1_Dep, Var1_Dep2, Var1_Dep3, Var2_Dep, Var2_Dep2, Var2_Dep3]

model_dict_DF = []
det_DF = []
corr_DF = []

for i in range(len(var_ind)):
  var_ind[i] = var_ind[i].fillna(method = 'bfill')
  var_ind[i] = var_ind[i].fillna(method = 'ffill')
  var_dep[i] = var_dep[i].fillna(method = 'bfill')
  var_dep[i] = var_dep[i].fillna(method = 'ffill')
  model.fit(X = var_ind[i], y = var_dep[i])
  det_DF.append(model.score(var_ind[i], var_dep[i]))
  corr_DF.append(np.sqrt(model.score(var_ind[i], var_dep[i])))
  model_dict_DF.append(model.__dict__)
  x = model_dict_DF[i]
  print(x['coef_'].max(),'x + ', x['intercept_'])

print()
display(det_DF, corr_DF)

0.4191589624896137 x +  65.19396569165667
266.1383615293329 x +  3250.8620770699536
2.9600116851153064 x +  103.99381950158191
0.35890393570477713 x +  77.93863112898893
-1.4565081397814352 x +  3221.882753860752
0.1576472520479008 x +  146.66605048954722



[0.13294744015038595,
 0.0011504841128776588,
 0.021911342165459424,
 0.12977675622943008,
 0.0003889329330216684,
 0.023680390787326755]

[0.36461958278510764,
 0.033918787019550964,
 0.14802480253477598,
 0.3602454111150204,
 0.019721382634634632,
 0.15388434224223968]

In [ ]:
df_1 = []
df_1 = pd.DataFrame(df_1)
df_1['Determinacion'] = det_DF[0:6]
df_1['Correlacion'] = corr_DF[0:6]

df_2 = []
df_2 = pd.DataFrame(df_2)
df_2['Determinacion'] = det_DF[6:]
df_2['Correlacion'] = corr_DF[6:]

Dist = []
Dist = pd.DataFrame(Dist)
Dist['Determinacion'] = det_DF
Dist['Correlacion'] = corr_DF

display("Para filtro 1",df_1,"Para filtro 2",df_2, "Total", Dist)

'Para filtro 1'

,Determinacion,Correlacion
0,0.132947,0.364620
1,0.001150,0.033919
2,0.021911,0.148025
3,0.129777,0.360245
4,0.000389,0.019721
5,0.023680,0.153884


'Para filtro 2'

,Determinacion,Correlacion


'Total'

,Determinacion,Correlacion
0,0.132947,0.364620
1,0.001150,0.033919
2,0.021911,0.148025
3,0.129777,0.360245
4,0.000389,0.019721
5,0.023680,0.153884


##Creación de modelos para Girona

In [ ]:
#Entire home/apt
Vars1_Indep= gir_1[['host_response_rate', 'price', 'number_of_reviews','review_scores_rating']]
Var1_Dep= gir_1['host_acceptance_rate']

Vars1_Indep2= gir_1[['availability_365','host_response_rate', 'number_of_reviews', 'accommodates']]
Var1_Dep2= gir_1['price']

Vars1_Indep3= gir_1[['number_of_reviews','review_scores_cleanliness', 'review_scores_communication']]
Var1_Dep3= gir_1['host_acceptance_rate']

Vars2_Indep= gir_1[['host_response_rate', 'price', 'number_of_reviews','review_scores_rating']]
Var2_Dep= gir_1['host_acceptance_rate']

Vars2_Indep2= gir_1[['availability_365','host_response_rate', 'number_of_reviews', 'accommodates']]
Var2_Dep2= gir_1['price']

Vars2_Indep3= gir_1[['number_of_reviews','review_scores_cleanliness', 'review_scores_communication']]
Var2_Dep3= gir_1['host_acceptance_rate']

In [ ]:
var_ind = [Vars1_Indep, Vars1_Indep2, Vars1_Indep3, Vars2_Indep, Vars2_Indep2, Vars2_Indep3]
var_dep = [Var1_Dep, Var1_Dep2, Var1_Dep3, Var2_Dep, Var2_Dep2, Var2_Dep3]

model_dict_gir = []
det_gir = []
corr_gir = []

for i in range(len(var_ind)):
  var_ind[i] = var_ind[i].fillna(method = 'bfill')
  var_ind[i] = var_ind[i].fillna(method = 'ffill')
  var_dep[i] = var_dep[i].fillna(method = 'bfill')
  var_dep[i] = var_dep[i].fillna(method = 'ffill')
  model.fit(X = var_ind[i], y = var_dep[i])
  det_gir.append(model.score(var_ind[i], var_dep[i]))
  corr_gir.append(np.sqrt(model.score(var_ind[i], var_dep[i])))
  model_dict_gir.append(model.__dict__)
  x = model_dict_gir[i]
  print(x['coef_'].max(),'x + ', x['intercept_'])

print()
display(det_gir, corr_gir)

0.36759062357488276 x +  65.70308138175739
33.60992204776545 x +  221.74459861853907
0.001342408721966157 x +  99.68665274988055
0.36759062357488276 x +  65.70308138175739
33.60992204776545 x +  221.74459861853907
0.001342408721966157 x +  99.68665274988055



[0.08008014967017874,
 0.02027521599697002,
 0.0001603276675150589,
 0.08008014967017874,
 0.02027521599697002,
 0.0001603276675150589]

[0.2829843629428643,
 0.14239106712490787,
 0.012662056211968848,
 0.2829843629428643,
 0.14239106712490787,
 0.012662056211968848]

In [ ]:
GIR_1 = []
GIR_1 = pd.DataFrame(GIR_1)
GIR_1['Determinacion'] = det_gir[0:6]
GIR_1['Correlacion'] = corr_gir[0:6]

GIR_2 = []
GIR_2 = pd.DataFrame(GIR_2)
GIR_2['Determinacion'] = det_gir[6:]
GIR_2['Correlacion'] = corr_gir[6:]

GIR = []
GIR = pd.DataFrame(GIR)
GIR['Determinacion'] = det_gir
GIR['Correlacion'] = corr_gir

display("Para filtro 1",GIR_1,"Para filtro 2",GIR_2, "Total", GIR)

'Para filtro 1'

,Determinacion,Correlacion
0,0.080080,0.282984
1,0.020275,0.142391
2,0.000160,0.012662
3,0.080080,0.282984
4,0.020275,0.142391
5,0.000160,0.012662


'Para filtro 2'

,Determinacion,Correlacion


'Total'

,Determinacion,Correlacion
0,0.080080,0.282984
1,0.020275,0.142391
2,0.000160,0.012662
3,0.080080,0.282984
4,0.020275,0.142391
5,0.000160,0.012662
